# Análise Completa dos Resultados

**Execute este notebook APÓS rodar `main.py`**

Aqui você vai:
- Ver resultados das 3 abordagens lado a lado
- Comparar métricas
- Analisar matrizes de confusão
- Decidir qual modelo é melhor para seu caso

In [ ]:
import sys
sys.path.append('..')

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from PIL import Image

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_style('whitegrid')

## 1. Carregar Resultados Salvos

Se você rodou `main.py`, os resultados estão salvos em `results/`

In [ ]:
results_dir = Path('../results')

# Verificar se resultados existem
if not results_dir.exists():
    print("⚠️ Pasta 'results/' não encontrada!")
    print("Execute 'python main.py' primeiro.")
else:
    print("✓ Resultados encontrados!")
    print(f"\nArquivos disponíveis:")
    for file in sorted(results_dir.glob('*.png')):
        print(f"  - {file.name}")

## 2. Visualizar Todas as Matrizes de Confusão

Comparação lado a lado das 3 abordagens

In [ ]:
# Carregar imagens das matrizes de confusão
cm_files = [
    'confusion_matrix_embedding.png',
    'confusion_matrix_finetuned.png',
    'confusion_matrix_llm.png'
]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, cm_file in zip(axes, cm_files):
    cm_path = results_dir / cm_file
    if cm_path.exists():
        img = Image.open(cm_path)
        ax.imshow(img)
        ax.axis('off')
    else:
        ax.text(0.5, 0.5, f'❌ {cm_file}\nnão encontrado', 
                ha='center', va='center', fontsize=12)
        ax.axis('off')

plt.tight_layout()
plt.show()

### 🔍 Análise das Matrizes

**O que observar:**

1. **Diagonal principal** (verde/azul escuro):
   - Quanto mais escuro, melhor (mais acertos)
   - Compare: qual modelo tem diagonal mais forte?

2. **Confusões comuns** (fora da diagonal):
   - Células claras = poucas confusões ✓
   - Células escuras = muitas confusões ✗
   - Os 3 modelos erram nos mesmos lugares?

3. **Classes problemáticas**:
   - Linha com muitos erros = modelo não aprendeu essa classe
   - Geralmente classes minoritárias (ex: surprise)

## 3. Comparação de Métricas

Gráfico gerado pelo `main.py`

In [ ]:
metrics_img = results_dir / 'metrics_comparison.png'

if metrics_img.exists():
    img = Image.open(metrics_img)
    plt.figure(figsize=(14, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("❌ metrics_comparison.png não encontrado")

### 🎯 Interpretação

**Perguntas para responder:**

1. **Qual modelo tem MAIOR Accuracy?**
   - Esperado: Fine-tuned DistilBERT

2. **Diferença é significativa?**
   - < 5% de diferença: Modelos equivalentes
   - 5-10%: Diferença moderada
   - > 10%: Diferença significativa

3. **Vale o trade-off?**
   - Fine-tuning é melhor, mas precisa treinar
   - Embedding é mais rápido, mas menos preciso
   - LLM é flexível, mas tem custo de API

## 4. Comparação de Tempo de Inferência

In [ ]:
time_img = results_dir / 'inference_time_comparison.png'

if time_img.exists():
    img = Image.open(time_img)
    plt.figure(figsize=(12, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("❌ inference_time_comparison.png não encontrado")

### ⚡ Trade-off: Velocidade vs Precisão

**Cenários de uso:**

| Cenário | Volume | Latência | Modelo Recomendado |
|---------|--------|----------|--------------------|
| Filtro de spam | 100k emails/dia | < 100ms | **Embedding** |
| Análise de sentimento | 1k posts/dia | < 1s | **Fine-tuned** |
| Chatbot | 10 msgs/min | < 5s | **LLM** |
| Análise batch | Offline | Não importa | **Fine-tuned** |

**Calcular custo:**
```
1 milhão de predições/dia:

Embedding:  1M × 0.5s  = 500,000s  = 139 horas (paralelo: ~6h com 24 cores)
Fine-tuned: 1M × 2.3s  = 2,300,000s = 639 horas (paralelo: ~27h com 24 cores)
LLM:        1M × 5.7s  = 5,700,000s = 1583 horas + custo API
```

## 5. Distribuição de Classes

In [ ]:
dist_files = [
    'class_distribution_train.png',
    'class_distribution_test.png'
]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, dist_file in zip(axes, dist_files):
    dist_path = results_dir / dist_file
    if dist_path.exists():
        img = Image.open(dist_path)
        ax.imshow(img)
        ax.axis('off')
    else:
        ax.text(0.5, 0.5, f'❌ {dist_file}\nnão encontrado', 
                ha='center', va='center', fontsize=12)
        ax.axis('off')

plt.tight_layout()
plt.show()

### 📊 Impacto do Desbalanceamento

**Observações:**

1. **Classes majoritárias** (joy, sadness):
   - Modelo aprende melhor (mais exemplos)
   - Precision e Recall altos

2. **Classes minoritárias** (surprise, love):
   - Modelo tem dificuldade (poucos exemplos)
   - Precision e Recall baixos

**Soluções para desbalanceamento:**
- Coletar mais dados das classes minoritárias
- Usar class weights no treinamento
- Oversampling (duplicar exemplos raros)
- Undersampling (reduzir exemplos comuns)
- Usar métricas weighted (já implementado)

## 6. Tabela Resumo Customizada

Vamos criar nossa própria tabela de comparação

In [ ]:
# Dados fictícios - SUBSTITUA pelos seus resultados reais
results = {
    'Modelo': ['Embedding + KNN', 'Fine-tuned DistilBERT', 'LLM (Gemini)'],
    'Accuracy': [0.75, 0.88, 0.82],
    'Precision': [0.73, 0.86, 0.80],
    'Recall': [0.72, 0.85, 0.79],
    'F1-Score': [0.72, 0.85, 0.80],
    'Tempo (s)': [0.5, 2.3, 5.7],
    'Treino Necessário': ['Não', 'Sim', 'Não'],
    'Custo': ['Grátis', 'GPU (treino)', 'API calls']
}

df_results = pd.DataFrame(results)

# Estilizar tabela
def highlight_max(s):
    if s.name in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
        is_max = s == s.max()
        return ['background-color: lightgreen' if v else '' for v in is_max]
    elif s.name == 'Tempo (s)':
        is_min = s == s.min()
        return ['background-color: lightgreen' if v else '' for v in is_min]
    return ['' for _ in s]

styled_df = df_results.style.apply(highlight_max)
styled_df

## 7. Decisão: Qual Modelo Escolher?

### Árvore de Decisão

```
1. Você tem GPU para treinar?
   ├─ SIM → Fine-tuned DistilBERT ⭐
   └─ NÃO → Próxima pergunta

2. Precisa de MÁXIMA precisão?
   ├─ SIM → LLM (Gemini)
   └─ NÃO → Próxima pergunta

3. Volume muito alto (>100k/dia)?
   ├─ SIM → Embedding + KNN
   └─ NÃO → Fine-tuned (treinar na CPU)
```

### Recomendações Práticas

**Para Produção:**
- 🥇 Fine-tuned DistilBERT (melhor custo-benefício)
- 🥈 Embedding + KNN (se volume muito alto)
- 🥉 LLM (se não pode treinar modelos)

**Para Protótipo Rápido:**
- 🥇 Embedding + KNN (mais rápido de implementar)
- 🥈 LLM (se tiver API key)
- 🥉 Fine-tuned (demora para treinar)

**Para Aprendizado:**
- 🥇 Fine-tuned (aprende mais conceitos)
- 🥈 Todos os 3 (comparar abordagens)
- 🥉 Embedding (mais simples)

## 8. Próximos Passos

### Para Melhorar os Resultados:

1. **Mais dados**
   - Aumentar MAX_SAMPLES
   - Coletar mais dados das classes minoritárias

2. **Mais treinamento (Fine-tuning)**
   - Aumentar EPOCHS (3 → 5)
   - Ajustar LEARNING_RATE

3. **Otimizar hiperparâmetros**
   - KNN: testar k=3, 5, 7, 10
   - Fine-tuning: batch_size, learning_rate
   - LLM: few-shot examples

4. **Análise de erros**
   - Ver notebook `error_analysis.ipynb`
   - Entender padrões nos erros
   - Melhorar dados baseado nos erros

### Para Entender Melhor:

1. **Análise de erros**: `error_analysis.ipynb`
2. **Teste interativo**: `interactive_playground.ipynb`
3. **Análise por classe**: Criar notebook customizado

### Para Produção:

1. **Salvar modelo escolhido**
2. **Criar API** (FastAPI, Flask)
3. **Monitorar performance** em produção
4. **Re-treinar periodicamente** com novos dados